In [1]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from tensorflow import keras
from tqdm import tqdm

## Считаем препроцессные данные из прошлого ноутбучка

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
interactions_df = pd.read_csv('/content/drive/MyDrive/ITMO/MTS/interactions_processed_kion.csv')
users_df = pd.read_csv('/content/drive/MyDrive/ITMO/MTS/users_processed_kion.csv')
items_df = pd.read_csv('/content/drive/MyDrive/ITMO/MTS/items_processed_kion.csv')

In [44]:
interactions_df['last_watch_dt'] = pd.to_datetime(interactions_df['last_watch_dt'], format='%Y-%m-%d')
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72
1,699317,1659,2021-05-29,8317,100
2,656683,7107,2021-05-09,10,0
3,864613,7638,2021-07-05,14483,100
4,964868,9506,2021-04-30,6725,100


In [45]:
items_df = items_df.rename(columns = {'id' : 'item_id'})

## Готовим фичи пользователей

Посмотрим, какие фичи в датасете фильмов являются категориальными и закодируем их с помощью one-hot encoding.

In [46]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом, 
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_income_unknown,sex_F,sex_M,sex_sex_unknown,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


## Готовим фичи айтемов

Кодируем их точно так же - one-hot'ом.

In [47]:
item_cat_feats = ['content_type', 'release_year_cat',
                  'for_kids', 'age_rating', 
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1) 

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,directors_ярив хоровиц,directors_ярон зильберман,directors_ярополк лапшин,directors_ярослав лупий,"directors_ярроу чейни, скотт моужер",directors_ясина сезар,directors_ясуоми умэцу,"directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори",directors_ёлкин туйчиев,directors_ён сан-хо
0,10711,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Подготовим данный для temporal слоя

In [48]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


In [49]:
max_date = interactions_df['last_watch_dt'].max()

train = interactions_df[interactions_df['last_watch_dt'] < max_date - pd.Timedelta(days=7)].copy()
test = interactions_df[interactions_df['last_watch_dt'] >= max_date - pd.Timedelta(days=7)].copy()

# Избавимся от пользователей с историей просмотров < 10
train_count = train.groupby('user_id')['item_id'].count()
valid_users = train_count[train_count > 5].index

train = train[train['user_id'].isin(valid_users)]

cold_users = set(test['user_id']) - set(train['user_id'])

# Отбрасываем холодных пользователей
test.drop(test[test['user_id'].isin(cold_users)].index, inplace=True)

In [50]:
common_users = set(train.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(train.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

train = train[train.item_id.isin(common_items)]
train = train[train.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

64838
6868


In [51]:
user_id_to_uid = {user_id:uid for uid, user_id in enumerate(np.sort(train.user_id.unique()))}
item_id_to_iid = {item_id:iid+1 for iid, item_id in enumerate(np.sort(train.item_id.unique()))}

user_id_to_uid_inv = {uid:user_id for user_id, uid in user_id_to_uid.items()}
item_id_to_idx_inv = {iid:item_id for item_id, iid in item_id_to_iid.items()}

In [52]:
# Получаем максимальную длину взаимодействий юзеров
MAX_LEN = train.user_id.value_counts().values[0]

# Получаем пользовательский взаимодействия, отсортированные по времени
train.sort_values(by='last_watch_dt', inplace=True)
user_temp_interactions = train.groupby('user_id')['item_id'].apply(list).values

In [53]:
# Добавляем паддинги
user_temp_interactions = keras.utils.pad_sequences(user_temp_interactions, maxlen=MAX_LEN)
user_temp_interactions = np.expand_dims(user_temp_interactions, axis=2)

In [55]:
items_ohe_df = items_ohe_df[items_ohe_df['item_id'].isin(train['item_id'].unique())]
users_ohe_df = users_ohe_df[users_ohe_df['user_id'].isin(train['user_id'].unique())]

# items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("item_id")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

In [56]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных 
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0) # возвращаем ноль, если лосс отрицательный
 
    return loss

Попробуйте другие лоссы, например, BPR Triplet loss

## Генератор и семплирование

Сделаем простой генератор. Он будет брать рандромного юзера, и два разных айтема - хороший пример и плохой:
- хорошим примером будет тот айтем, который был взят из датасета взаимодействий в соответствии с распределением просмотренных айтемов для этого юзера;
- а плохим айтемом будет просто любой другой _случайный айтем_*


In [57]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0]-1)
            # id хорошего айтема
            inter = np.squeeze(interactions[uid_i][interactions[uid_i] > 0])
            pos_i = np.random.choice(inter)
            # id плохого айтема
            neg_i = np.random.choice(items.index)
            # фичи юзера
            uid_meta.append(users.loc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions[uid_i])
            # фичи хорошего айтема
            pos.append(items.loc[pos_i])
            # фичи плохого айтема
            neg.append(items.loc[neg_i])
            
        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


### Сделайте генаратор, который будет использовать информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования

In [ ]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.shape[1], ) 
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )


print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"MAX_SEQUENCE_SHAPE: {MAX_LEN}")

In [63]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    
    # полносвязный слой
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                               kernel_regularizer=keras.regularizers.l2(1e-6),
                               activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    # делаем residual connection - складываем два слоя, 
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)
    
    add = keras.layers.Add()([layer_1, layer_2])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)
    
    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)

    inp_temp_interaction = keras.layers.Input(shape=(MAX_LEN, 1))

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)
                              
    temporal_layer = keras.layers.LSTM(N_FACTORS, activation='elu', use_bias=False)(inp_temp_interaction)
    # flutten = keras.layers.Flatten()(temporal_layer)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)
    

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])
    
    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, temporal_layer])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)
    
    return keras.models.Model([inp_meta, inp_temp_interaction], out)

# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_temporal_in = keras.layers.Input(shape=(MAX_LEN, 1))

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_temporal_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_temporal_in, pos_in, neg_in], res)

### Попробуйте передать текстовые признаки в модель

In [64]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")

In [65]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [66]:
model.fit(generator(items=items_ohe_df,
                    users=users_ohe_df.drop(["user_id"], axis=1), 
                    interactions=user_temp_interactions,
                    batch_size=128), 
          steps_per_epoch=100, 
          epochs=15, 
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

Epoch 1/15
100/100 [==============================] - 114s 1s/step - loss: 0.2304 - lr: 0.0010
Epoch 2/15
100/100 [==============================] - 99s 995ms/step - loss: 0.1713 - lr: 0.0010
Epoch 3/15
100/100 [==============================] - 96s 959ms/step - loss: 0.1656 - lr: 0.0010
Epoch 4/15
100/100 [==============================] - 95s 949ms/step - loss: 0.1548 - lr: 0.0010
Epoch 5/15
100/100 [==============================] - 96s 960ms/step - loss: 0.1586 - lr: 0.0010
Epoch 6/15
100/100 [==============================] - 96s 964ms/step - loss: 0.1462 - lr: 0.0010
Epoch 7/15
100/100 [==============================] - 95s 949ms/step - loss: 0.1451 - lr: 0.0010
Epoch 8/15
100/100 [==============================] - 96s 961ms/step - loss: 0.1412 - lr: 0.0010
Epoch 9/15
100/100 [==============================] - 97s 971ms/step - loss: 0.1391 - lr: 0.0010
Epoch 10/15
100/100 [==============================] - 95s 955ms/step - loss: 0.1379 - lr: 0.0010
Epoch 11/15
100/100 [==========

In [77]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = user_temp_interactions[rand_uid]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.loc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1), 
                        np.array(user_interaction_vec).reshape(1, -1)])

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
from sklearn.metrics.pairwise import euclidean_distances as ED

ED(user_vec, item_vec)

1/1 [==============================] - 0s 34ms/step


array([[2.0378084]], dtype=float32)